<a href="https://colab.research.google.com/github/farrukh602/smollm-fine-tuning/blob/main/sft_smollm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qU trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from huggingface_hub import login
login()

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTJForCausalLM, GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
model_name="HuggingFaceTB/SmolLM2-135M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# setupt the chat format
model, tokenizer = setup_chat_format(model, tokenizer)

# setup the name for the finetune
finetune_name="smollm2-sft-test1"
finetune_tags=['smol-course','module_1']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [11]:
# trying the base model which does not have a chat template yet
prompt="What is haiku in Japenese literature"
messages=[{"role":"user","content":"prompt"}]
formated_prompt=tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
formated_prompt

'<|im_start|>user\nprompt<|im_end|>\n<|im_start|>assistant\n'

In [16]:
inputs=tokenizer(formated_prompt,return_tensors="pt")
outputs=model.generate(**inputs, max_new_tokens=100)
outputs

tensor([[    1,  4093,   198, 41253,     2,   198,     1,   520,  9531,   198,
           520,  9531,   198,   520,  9531,   198,   520,  9531,   198,   520,
          9531,   198,   520,  9531,   198,   520,  9531,   198,   520,  9531,
           198,   520,  9531,   198,   520,  9531,   198,   520,  9531,   198,
           520,  9531,   198,   520,  9531,   198,   520,  9531,   198,   520,
          9531,   198,   520,  9531,   198,   520,  9531,   198,   520,  9531,
           198,   520,  9531,   198,   520,  9531,   198,   520,  9531,   198,
           520,  9531,   198,   520,  9531,   198,   520,  9531,   198,   520,
          9531,   198,   520,  9531,   198,   520,  9531,   198,   520,  9531,
           198,   520,  9531,   198,   520,  9531,   198,   520,  9531,   198,
           520,  9531,   198,   520,  9531,   198,   520,  9531,   198,   520]])

In [19]:
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|im_start|>user
prompt<|im_end|>
<|im_start|>assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
assistant
ass


### Data set prepration
- For SFT, dataset should be structured with input-output pairs.
- Each input is a prompt and the output is the expected response from the model

In [25]:
# TRL formats input messages based on the model's chat template
# messages=[{"role":, "conent":}]
from datasets import load_dataset
smoltalk=load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
gsm8k=load_dataset("openai/gsm8k",name='socratic')

In [30]:
# Configure sft
sft_config=SFTConfig(output_dir="/content/drive/MyDrive/smollm-fine-tuning/trained_models",
                     max_steps=1000,
                     per_device_train_batch_size=10,
                     learning_rate=5e-5,
                     logging_steps=10,
                     save_steps=100,
                     evaluation_strategy='steps',
                     eval_steps=50,
                     hub_model_id=finetune_name)
trainer=SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=smoltalk['train'],
    tokenizer=tokenizer,
    eval_dataset=smoltalk['test']
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-30-369f332e5b41>:11: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer=SFTTrainer(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

### Model Training

In [31]:
trainer.train()
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
50,1.108600,1.126347
100,1.048500,1.081968
150,1.067600,1.060746
200,1.083900,1.042642
250,0.911000,1.037163
300,0.871700,1.034717
350,0.825800,1.029841
400,0.850400,1.024226
450,0.867900,1.017557
500,0.735900,1.039349


In [34]:
!cp -r /content/smollm-fine-tuning/wandb /content/drive/MyDrive/smollm-fine-tuning/trained_models


In [35]:
trainer.push_to_hub(tags=finetune_tags)


training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1734934368.f534e42ab4c2.632.0:   0%|          | 0.00/32.5k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sfarrukh/smollm2-sft-test1/commit/4deee9548fe6f46c77aebcff7fb354aaa3a9d0f9', commit_message='End of training', commit_description='', oid='4deee9548fe6f46c77aebcff7fb354aaa3a9d0f9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sfarrukh/smollm2-sft-test1', endpoint='https://huggingface.co', repo_type='model', repo_id='sfarrukh/smollm2-sft-test1'), pr_revision=None, pr_num=None)

In [41]:
from transformers import pipeline

question = "who are you"
generator = pipeline("text-generation", model="sfarrukh/smollm2-sft-test1", device="cuda")
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)
# print(output["generated_text"])


Device set to use cuda


In [42]:
print(output[0]['generated_text'])

Hello! How can I help you today?
I'm a language model and I'm looking for help with a new project. Can you help me with the translation of a sentence?
I'm a language model and I'm looking for help with a new project. Can you help me with the translation of a sentence?
The sentence is "I'm a good person." Can you translate it to English?
I'm a language model and I'm looking for help with a new project. Can you help me with the translation of a sentence?
The sentence is "I'm a good person." Can you translate it to English
